In [61]:
#%%capture output
!pip install brian2
!pip install -q condacolab
import condacolab
condacolab.install()
!conda create -y -n opensim_env python=3.10
!source /usr/local/etc/profile.d/conda.sh && conda activate opensim_env && conda install -y -c opensim-org opensim pandas
!wget https://raw.githubusercontent.com/MathieuCharbonnier/Loop/refs/heads/main/activation.py
!wget https://raw.githubusercontent.com/MathieuCharbonnier/Loop/refs/heads/main/muscle_sim.py
!wget https://raw.githubusercontent.com/MathieuCharbonnier/Loop/refs/heads/main/neural_simulations.py
!wget https://raw.githubusercontent.com/MathieuCharbonnier/Loop/refs/heads/main/plot_time_series.py



✨🍰✨ Everything looks OK!
Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/opensim_env

  added / updated specs:
    - python=3.10


The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge 
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu 
  bzip2              conda-forge/linux-64::bzip2-1.0.8-h4bc722e_7 
  ca-certificates    conda-forge/linux-64::ca-certificates-2025.1.31-hbcca054_0 
  ld_impl_linux-64   conda-forge/linux-64::ld_impl_linux-64-2.43-h712a8e2_4 
  libexpat           conda-forge/linux-64::libexpat-2.7.0-h5888daf_0 
  libffi             conda-forge/linux-64::libffi-3.4.6-h2dba641_1 
  libgcc             conda-forge/li

In [1]:
%%capture output
!unzip Model.zip

In [79]:
%load_ext autoreload
%autoreload 2

from brian2 import *
import numpy as np
import pandas as pd
import os
import subprocess
import tempfile
import json
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
from collections import defaultdict

from plot_time_series import plot_times_series
from neural_simulations import run_neural_simulations
from activation import decode_spikes_to_activation


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
def RampHold(T_reaction, dt, v=0.2, t_ramp=0.1, t_hold=0.3, plot=False):

  time = np.arange(0, T_reaction, dt)

  stretch = np.piecewise(
      time,
      [time < t_ramp,
      (time >= t_ramp) & (time < t_hold),
      time >= t_hold],
      [lambda t: v * t,
      lambda t: v * t_ramp,
      lambda t: v * t_ramp - v * (t - t_hold)]
  )
  velocity = np.gradient(stretch, dt)
  if plot:
    plt.plot(time, stretch)
    plt.xlabel('Time (s)')
    plt.ylabel('Stretch (m)')
    plt.title('Ramp-Hold Stretch Profile')
    plt.show()
  return stretch, velocity

def SinusoidalStretch(dt,T_reaction, A=0.01, f=2, plot=False):

  time = np.arange(0, T_reaction, dt)
  stretch = A * np.sin(2 * np.pi * f * time)
  velocity = np.gradient(stretch, dt)
  if plot:
    plt.plot(time, stretch)
    plt.xlabel('Time (s)')
    plt.ylabel('Stretch (m)')
    plt.title('Sinusoidal Stretch Profile')
    plt.show()

  return stretch, velocity

def NoInput(dt, T_reaction):

  time = np.arange(0, T_reaction, dt)
  stretch = np.zeros_like(time)
  velocity = np.zeros_like(time)
  return stretch, velocity


In [82]:
t_stretching_afferent = 16*ms
t_motoneuron_muscle = 10*ms
t_reaction = t_stretching_afferent + t_motoneuron_muscle
n_loop = 1
dt = 0.1*ms

name_output_spikes = 'spikes.json'
name_output_muscle='df_muscle.csv'

neuron_population = {"Ia": 10, "II": 10, "exc": 10, "motor": 10}
muscle_name = "iliacus_r"
resting_length = 0.01
stretch, velocity = RampHold(t_reaction, dt, v=0.2, t_ramp=0.1, t_hold=0.3)

data_muscle = []
data_spikes = {
    "Ia": defaultdict(list),
    "II": defaultdict(list),
    "MN": defaultdict(list)
}
current_length=resting_length
for j in range(n_loop):

    spikes_all=run_neural_simulations(stretch, velocity, neuron_population,dt,t_reaction, w_run=500*uS, p_run=0.4 )
    for fiber_name, fiber_spikes in spikes_all.items():
      for neuron_id, spikes in fiber_spikes.items():
        data_spikes[fiber_name][neuron_id].extend(spikes)

    activations=decode_spikes_to_activation([value/second for key, value in spikes_all["MN"].items()], dt/second, t_reaction/second)
    mean_activation = np.mean(activations, axis=0)

    # Create temporary files for both input and output
    with tempfile.NamedTemporaryFile(suffix='.npy', delete=False) as input_tmp, \
        tempfile.NamedTemporaryFile(suffix='.npy', delete=False) as output_tmp:

        input_path = input_tmp.name
        output_path = output_tmp.name
        np.save(input_path, mean_activation)

        # Run the script in the other environment, passing both file paths
        process=subprocess.run([
            'conda', 'run', '-n', 'opensim_env', 'python', 'muscle_sim.py',
            '--dt', str(dt/second),
            '--T', str(t_reaction/second),
            '--muscle_name', muscle_name,
            '--resting_length', str(current_length),
            '--input_file', input_path,
            '--output_file', output_path
        ], capture_output=True, text=True)

        if process.stdout.strip():
            print("STDOUT:\n", process.stdout)

        if process.returncode == 0 and os.path.getsize(output_path) > 0:
            fiber_length=np.load(output_path)
            stretch=fiber_length/resting_length-1

            # Create batch data dictionary
            batch_data_muscle = {
              **{f'activation_{i}': activations[i] for i in range(neuron_population['motor'])},
              'activation_mean': mean_activation,
              'fiber_length': fiber_length,
              'stretch': stretch,
              'velocity': np.gradient(stretch)
            }
            data_muscle.append(pd.DataFrame(batch_data_muscle))
        else:
          raise RuntimeError(f'Problem in iteration {j}. STDERR: {process.stderr}')

        # Clean up temporary files
        os.unlink(input_path)
        os.unlink(output_path)


df_muscle = pd.concat(data_muscle)
df_muscle['Time']=df_muscle.index*dt/second
df_muscle.to_csv(name_output_muscle, index=False)

with open(name_output_spikes, "w") as f:
    json.dump(data_spikes, f, indent=4)

plot_times_series(name_output_spikes, name_output_muscle)

RuntimeError: Problem in iteration 0. STDERR: Traceback (most recent call last):
  File "/content/muscle_sim.py", line 52, in <module>
    parser.add_arguement('--L', type=float, required=True, help='initial fiber length')
AttributeError: 'ArgumentParser' object has no attribute 'add_arguement'. Did you mean: 'add_argument'?

ERROR conda.cli.main_run:execute(125): `conda run python muscle_sim.py --dt 0.0001 --T 0.026000000000000002 --muscle_name iliacus_r --resting_length 0.01 --input_file /tmp/tmpqez9onad.npy --output_file /tmp/tmpsb1zggn3.npy` failed. (See above for error)
